In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.optim.lr_scheduler import LambdaLR

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import random
import time

import adai_optim

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

/home/jukie/anaconda3/envs/py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('/home/jukie/Project/eeg-eye-state-classification-master/eeg_eye_state.csv', skiprows = [i for i in range(19)], header=None)

In [3]:
X = df.loc[:,0:13]
y = df.loc[:,14:]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1234)

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [6]:
print(f"Training Data has {len(X_train)} examples")
print(f"Testing Data has {len(X_test)} examples")

Training Data has 12717 examples
Testing Data has 2245 examples


In [7]:
y_train = y_train.to_numpy()


In [8]:
y_test =y_test.to_numpy()

In [9]:
#X_train = X_train.to_numpy()
#X_test = X_test.to_numpy()

In [10]:
## Train Data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = trainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
## Test Data    
class testData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = testData(torch.FloatTensor(X_test),
                    torch.FloatTensor(y_test))

In [11]:
# Create DataLoaders for torch
BATCH_SIZE = 64
LEARNING_RATE=0.001
EPOCHS=50

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE,)

In [12]:
class eca_layer(nn.Module):
    """Constructs a ECA module.

    Args:
        channel: Number of channels of the input feature map
        k_size: Adaptive selection of kernel size
    """
    def __init__(self, channel, k_size=3):
        super(eca_layer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size - 1) // 2, bias=False) 
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # feature descriptor on the global spatial information
        y = self.avg_pool(x)

        # Two different branches of ECA module
        y = self.conv(y.squeeze(-1).transpose(-1, -2)).transpose(-1, -2).unsqueeze(-1)

        # Multi-scale information fusion
        y = self.sigmoid(y)

        return x * y.expand_as(x)

In [13]:
class GRUnet1(nn.Module):
    def __init__(self):
        super(GRUnet1,self).__init__()
        self.embed = nn.Sequential(
            nn.Linear(1,64),
            #nn.Dropout(p=0.3),
            nn.ReLU())
        self.gru1 = nn.LSTM(input_size=64,hidden_size=256,batch_first=True,bidirectional=True)
        self.gru2 = nn.LSTM(input_size=512,hidden_size=128,batch_first=True,bidirectional=True)
        self.gru3 = nn.GRU(input_size=256,hidden_size=32,batch_first=True,bidirectional=True)
        self.tanh = nn.Tanh()
        self.gru = nn.GRU(input_size=64,hidden_size=128,batch_first=True,bidirectional=True)

        self.conn1 = nn.Dropout(p=0.3)
        self.eca1 = eca_layer(64,3)
        self.eca2 = eca_layer(256,3)
        self.fc = nn.Linear(128, 1)
        self.flat = nn.Sequential(
            nn.Linear(3584,128),
            #nn.Dropout(p=0.5),
            nn.ReLU())
        self.head = nn.Sigmoid()

    def forward(self,x):
        x = torch.unsqueeze(x,2)
        x = self.embed(x)

        #x = self.eca1(x)
        x,n = self.gru1(x,None)
        x = self.tanh(x)
        x = self.conn1(x)
        x,n = self.gru2(x,None)
        x = self.tanh(x)
        #x,n = self.gru(x,None)
        x = self.conn1(x)
        #x,n = self.gru3(x,None)
        #x = self.conn1(x)
        #x = self.eca2(x)
        x = torch.flatten(x,1)
        x = self.flat(x)
        x = self.fc(x)
        x = self.head(x)
        return x

model = GRUnet1()

In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,980,679 trainable parameters


In [15]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=5e-4)
#optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
#optimizer = adai_optim.AdaiW(model.parameters(), lr=LEARNING_RATE, betas=(0.1, 0.99), eps=1e-03, weight_decay=5e-4)
#criterion = nn.BCEWithLogitsLoss()
criterion = nn.BCELoss()
#scheduler_1 = LambdaLR(optimizer, lr_lambda=lambda epoch: 0.001 * np.exp(-epoch / 10.0))
device = torch.device('cuda:0')

In [16]:
model = model.to(device)
criterion = criterion.to(device)

In [17]:
def accuracy(y_pred, y_test):
    #y_pred_tag = torch.round(torch.sigmoid(y_pred))
    y_pred_tag = torch.round(y_pred)

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [18]:
#model.train()
for e in range(1, 101):
    epoch_loss = 0
    epoch_acc = 0
    tepoch_loss = 0
    tepoch_acc = 0
    model.train()
    for X_batch, y_batch in train_loader:
        
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        #y_pred = torch.argmax(y_pred, dim=1)
        #y_pred = torch.FloatTensor(y_pred)
        #y_pred = y_pred.to(device)
        #y_pred = y_pred.float()
        loss = criterion(y_pred, y_batch)
        acc = accuracy(y_pred, y_batch)
        #loss.requires_grad_()
        loss.backward()
        optimizer.step()
        #scheduler_1.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    model.eval()
    for X_batch, y_batch in test_loader:
        
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        
        y_pred = model(X_batch)
        #y_pred = torch.argmax(y_pred, dim=1)
        #y_pred = torch.FloatTensor(y_pred)
        #y_pred = y_pred.to(device)
        #y_pred = y_pred.float()
        tloss = criterion(y_pred, y_batch)
        tacc = accuracy(y_pred, y_batch)
                
        tepoch_loss += tloss.item()
        tepoch_acc += tacc.item()
        
    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')    
    print(f'Epoch {e+0:03}: | tLoss: {tepoch_loss/len(test_loader):.5f} | tAcc: {tepoch_acc/len(test_loader):.3f}')

Epoch 001: | Loss: 0.68268 | Acc: 55.467
Epoch 001: | tLoss: 0.66670 | tAcc: 58.972
Epoch 002: | Loss: 0.66165 | Acc: 57.779
Epoch 002: | tLoss: 0.69637 | tAcc: 60.222
Epoch 003: | Loss: 0.65057 | Acc: 59.724
Epoch 003: | tLoss: 0.69040 | tAcc: 58.722
Epoch 004: | Loss: 0.63069 | Acc: 62.462
Epoch 004: | tLoss: 0.78591 | tAcc: 58.417
Epoch 005: | Loss: 0.59393 | Acc: 66.578
Epoch 005: | tLoss: 0.84207 | tAcc: 59.194
Epoch 006: | Loss: 0.56280 | Acc: 70.357
Epoch 006: | tLoss: 0.80216 | tAcc: 61.611
Epoch 007: | Loss: 0.53729 | Acc: 72.432
Epoch 007: | tLoss: 0.90183 | tAcc: 60.361
Epoch 008: | Loss: 0.51614 | Acc: 73.879
Epoch 008: | tLoss: 1.00464 | tAcc: 59.833
Epoch 009: | Loss: 0.50384 | Acc: 74.573
Epoch 009: | tLoss: 0.85168 | tAcc: 61.639
Epoch 010: | Loss: 0.49180 | Acc: 75.538
Epoch 010: | tLoss: 0.92918 | tAcc: 61.111
Epoch 011: | Loss: 0.48666 | Acc: 75.879
Epoch 011: | tLoss: 0.97377 | tAcc: 60.333
Epoch 012: | Loss: 0.47323 | Acc: 76.302
Epoch 012: | tLoss: 1.05393 | tAcc:

In [19]:
class accData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

acc_data = accData(torch.FloatTensor(X_test))

In [20]:
acc_loader = DataLoader(dataset=acc_data, batch_size=1)

In [21]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in acc_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        #y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [23]:
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.65      0.57      0.61      1233
           1       0.54      0.62      0.58      1012

    accuracy                           0.59      2245
   macro avg       0.60      0.60      0.59      2245
weighted avg       0.60      0.59      0.59      2245

